In [12]:
# Import dependencies
import pandas as pd
import gmaps
import requests
# Import API key
from configure import g_key

In [13]:
# Read and load the cities database
city_data_df = pd.read_csv('weather_data\cities.csv')
# Get data type
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [15]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [20]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4) #[max(0,temp) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Heatmap of percentage of humidity
locations= city_data_df[['Lat', 'Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level =1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the fig to plot data
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Heatmap of cloudiness
locations = city_data_df[['Lat', 'Lng']]
cloudiness = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# Heatmap of wind speed
locations = city_data_df[['Lat','Lng']]
wind = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30,31), zoom_level=1.5)
heat_layer= gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4 )

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 93


In [41]:
# Filter dataset to find cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Presidencia Roque Saenz Pena,-26.79,-60.44,70.29,79,8,7.02,AR,2019-11-06 03:35:00
13,13,Avarua,-21.21,-159.78,73.40,83,100,9.17,CK,2019-11-06 03:35:18
15,15,Georgetown,6.80,-58.16,77.00,94,20,4.70,GY,2019-11-06 03:35:19
18,18,Butaritari,3.07,172.79,84.83,71,65,9.66,KI,2019-11-06 03:33:08
19,19,Meulaboh,4.14,96.13,81.30,75,83,8.97,ID,2019-11-06 03:35:23


In [39]:
preferred_cities_df.count()

City_ID       192
City          192
Lat           192
Lng           192
Max Temp      192
Humidity      192
Cloudiness    192
Wind Speed    192
Country       192
Date          192
dtype: int64

In [38]:
# Create dataframe called hotel_df to store hotel names along with city, country, max temperature and coordinates
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Presidencia Roque Saenz Pena,AR,70.29,-26.79,-60.44,
13,Avarua,CK,73.40,-21.21,-159.78,
15,Georgetown,GY,77.00,6.80,-58.16,
18,Butaritari,KI,84.83,3.07,172.79,
19,Meulaboh,ID,81.30,4.14,96.13,


In [42]:
# Set the parameters to search for a hotel
params = {'radius': 5000,
          'type': 'lodging',
          'key': g_key}

In [46]:
# Iterate through the Dataframe
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row['Lat']
    lng = row['Lng']
    # Add the latitude and longitude to the params dictionary as values to location key
    params['location'] = f'{lat},{lng}'
    # Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')
        pass

    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Presidencia Roque Saenz Pena,AR,70.29,-26.79,-60.44,Hotel Aconcagua
13,Avarua,CK,73.40,-21.21,-159.78,Paradise Inn
15,Georgetown,GY,77.00,6.80,-58.16,Kanhai's Center
18,Butaritari,KI,84.83,3.07,172.79,Isles Sunset Lodge
19,Meulaboh,ID,81.30,4.14,96.13,Kost Pria


In [48]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Presidencia Roque Saenz Pena,AR,70.29,-26.79,-60.44,Hotel Aconcagua
13,Avarua,CK,73.40,-21.21,-159.78,Paradise Inn
15,Georgetown,GY,77.00,6.80,-58.16,Kanhai's Center
18,Butaritari,KI,84.83,3.07,172.79,Isles Sunset Lodge
19,Meulaboh,ID,81.30,4.14,96.13,Kost Pria
23,Quang Ngai,VN,75.24,15.12,108.80,Central Hotel
26,Bluff,AU,82.92,-23.58,149.07,Bluff Hotel
27,Puerto Colombia,CO,82.40,10.99,-74.96,Hotel El Emigrante
28,Muriwai Beach,NZ,75.00,-36.82,174.45,Coast Retreat Muriwai - Golf and Surf Holiday
30,Busselton,AU,73.00,-33.64,115.35,Observatory Guest House


In [52]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [53]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [54]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))